## Interacting with the Batfish service


### Python Imports

In your Python program or in the Python shell you will need to import Pybatfish modules. The common imports are shown below. Depending upon your program you may need to add to this list.

In [1]:
>>> import pandas as pd
>>> from pybatfish.client.commands import *
>>> from pybatfish.datamodel import Edge, Interface
>>> from pybatfish.datamodel.answer import TableAnswer
>>> from pybatfish.datamodel.flow import (HeaderConstraints,
...                                       PathConstraints)
>>> from pybatfish.question import bfq, list_questions, load_questions

In [2]:
# Configure all pybatfish loggers to use WARN level
>>> import logging
>>> logging.getLogger('pybatfish').setLevel(logging.WARN)


### Sessions

The Batfish service may be running locally on your machine, or on a remote server. The first step in analyzing your configurations is setting up the connection to the Batfish service.

In [3]:
>>> bf_session.host = 'localhost'

### Load Questions from the Batfish server

In [4]:
>>> load_questions()

### Uploading configurations to Batfish for analysis

Batfish is designed to analyze a series of snapshots related to a network. This can be all of the devices in your network, or could be a subset of the devices; for example all of the devices in the your DC-ASH1 (data-center in Ashburn, VA).

So the first step in the process is to specify which network you want to operate on. 

In [5]:
>>> bf_set_network('example')

'example'

#### Working with snapshots

Now you are ready to create your first snapshot. Think of this as a collection of information about the devices in the network you are analyzing. 

#### Packaging snapshot data

Batfish expects configuration files to be organized according to a folder structure outlined below. The names in bold are keywords and must be used in your network. Your snapshot need not have all of these components, e.g., if you are not analyzing hosts or iptables configurations, the corresponding folders are not needed.

***
snapshot _[top-level folder]_

* **configs** _[folder with configurations for network devices]_
   * router1.cfg
   * router2.cfg
   * ....

* **hosts** _[folder with host configurations (e.g., pointers to their iptables files)]_
   * host1.json _[see below for the expected format]_
   * host2.json
   * ....

* **iptables** _[folder with iptables configuration files; host JSON files should point to these files]_
   * host1.iptables
   * host2.iptables
   * ....

* **batfish** _[miscellaneous information (not network device configuration) used by Batfish]_
   * isp_config.json _[configuration file used to model Internet Service Providers (ISPs), see below for expected format]_ 
***

See the [example snapshot](https://github.com/batfish/batfish/tree/master/networks) in the repository as an example. For convenience, this folder contains some extra files that are not used by Batfish.

When you supply the snapshot as a zipped file, the top-level folder (called "snapshot" above) should be part of the zip archive. 


##### Format for F5 BIG-IP configuration files

F5 BIG-IP configuration spans multiple individual files. Follow the instructions [here](https://github.com/batfish/batfish/wiki/Packaging-F5-Big-IP-configuration-for-analysis) to properly package them for analysis.


##### Format for Cumulus configuration files

Cumulus devices can be configured by editing individual files, such as `/etc/interfaces`, `/etc/frr.conf` or invoking the [Network Command-Line Utility (NCLU)](https://docs.cumulusnetworks.com/display/DOCS/Network+Command+Line+Utility+-+NCLU)

Currently, Batfish ONLY supports processing of the NCLU configuration output. To retrieve the Cumulus configuration in the NCLU format, issue `net show config commands` and save the output to a file.

If you are using the `BGP Unnumbered` feature on Cumulus devices, you will need to supply a [Layer1 topology file](#format-for-the-topology-file).


##### Format for host JSON files

The host JSON files contain basic information about the hosts attached to the network, including their names, a pointer to their iptables configuration file, and their interfaces. An example file is:

```json
{  
  "hostname" : "host1",  
  "iptablesFile" : "iptables/host1.iptables",  
  "hostInterfaces" : {  
    "eth0" : {  
      "name": "eth0",  
      "prefix" : "2.128.0.101/24"  
    }  
  }  
}
```

`iptables/host1.iptables` is the path relative to the snapshot where this host's iptables configuration can be found. iptables configuration files should be in the format that is generated by `iptables-save`.


##### Format for the topology file

Normally Batfish infers layer-3 interface adjacencies based on IP address configuration on interfaces.

For instance, if there are two interfaces in the network with IP assignments `192.168.1.1/24` and `192.128.1.2/24` respectively, Batfish will infer that these interfaces are adjacent.

However, you may override this behavior by supplying a layer-1 topology file. In this case, layer-3 adjacencies are computed by combining the supplied layer-1 adjacencies with layer-2 and layer-3 configuration to get a more accurate model. This is especially useful if IP addresses are reused across the network on interfaces that are not actually adjacent in practice.

Batfish can take the topology file in JSON format:

   The expected JSON file has a list of edge records, where each edge record has node and interface names of the two ends. See [this file](https://github.com/batfish/batfish/tree/master/networks/example/example_layer1_topology.json) for an example. Your file name should be `layer1_topology.json` for it to be considered by Batfish.


##### Format for ISP configuration json files

Batfish can model routers representing ISPs (and Internet) for a given network. The modeling is based on a json configuration file (`isp_config.json`), which tells Batfish about the interfaces on border routers which peer with the ISPs. An example file is:

```json
{
  "borderInterfaces": [
    {
      "borderInterface": {
        "hostname": "as2border1",
        "interface": "GigabitEthernet3/0"
      }
    }, {
      "borderInterface": {
        "hostname": "as2border2",
        "interface": "GigabitEthernet3/0"
      }
    }
  ],
  "filter": {
    "onlyRemoteAsns": [],
    "onlyRemoteIps": []
  }
}
```

Here `borderInterfaces` contains the list of interfaces on border routers which are meant to peer with the ISPs. `onlyRemoteAsns` (list of ASNs) and `onlyRemoteIps` (list of IPs) provide a way to apply additional filter by restricting to ISPs having specific ASNs or IPs. 

_Batfish will not try to model any ISP routers in the absence of this configuration file._

An example network with ISP modeling configuration is [here](https://github.com/batfish/batfish/tree/master/networks/example/live-with-isp).


#### Initializing a new snapshot

In [6]:
>>> SNAPSHOT_DIR = '../../networks/example'
>>> bf_init_snapshot(SNAPSHOT_DIR, name='snapshot0', overwrite=True)

'snapshot0'

Batfish supports a wide variety of devices, device types and configuration constructs. But it may not fully support your configuration files. To check on the status of the snapshot you just initialized, you can use the `bfq.initIssues` query.

In [7]:
>>> bfq.initIssues().answer()

,Nodes,Source_Lines,Type,Details,Line_Text,Parser_Context
0,['as1border1'],None,Convert warning (redflag),Could not determine update source for BGP neig...,None,None
1,['as1border1'],None,Convert warning (redflag),Could not determine update source for BGP neig...,None,None


#### Using an existing snapshot

If you have a previously initialized snapshot that you would like to work with, you do not need to re-initialize it.

In [8]:
>>> bf_set_network('example')

'example'

In [9]:
>>> bf_set_snapshot('snapshot0')

'snapshot0'

### Running Questions

Now that you have initialized a snapshot, you can query the Batfish service to retrieve information about the snapshot.

Let’s start with the basics. The Batfish service will return the list of questions

In [10]:
>>> len(list_questions())

64

In [11]:
>>> type(list_questions())

list

In [12]:
>>> list_questions()[0]

{'name': 'aaaAuthenticationLogin',
 'description': 'Returns nodes that do not require authentication on all virtual terminal lines.\n\nLists all nodes in the network for which there is a virtual terminal line that does not require authentication.',
 'tags': ['hygiene']}

To invoke any question, you use the form `bfq.<question_name>()`. This will create a question object.

In [13]:
>>> question = bfq.nodeProperties()

In [14]:
type(question)

pybatfish.question.bfq.nodeProperties

To send the query to the Batfish service and retrieve the results, you need to run:

In [15]:
>>> result = bfq.nodeProperties().answer()

In [16]:
>>> type(result)

pybatfish.datamodel.answer.table.TableAnswer

This will return information as JSON. For easy manipulation of the data, you can turn it into a Pandas dataframe.

In [17]:
>>> result = bfq.nodeProperties().answer().frame()

In [18]:
>>> type(result)



pandas.core.frame.DataFrame